# Live Demo Code for Connecting to PostgreSQL from Python

Notebook for your helpful reference - but the assignment still requires writing .py files!

In [ ]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 2.8MB/s 


In [ ]:
import psycopg2

In [ ]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

In [ ]:
# psycopg2.connect looks interesting, similar to sqlite3
help(psycopg2.connect)

Help on function connect in module psycopg2:

connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)
    Create a new database connection.
    
    The connection parameters can be specified as a string:
    
        conn = psycopg2.connect("dbname=test user=postgres password=secret")
    
    or using a set of keyword arguments:
    
        conn = psycopg2.connect(database="test", user="postgres", password="secret")
    
    Or as a mix of both. The basic connection parameters are:
    
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)
    
    Using the *connection_factory* parameter a different class or connections
    factory can be specified. It should be a callable object tak

In [ ]:
# Looks similar to sqlite3, but needs more info to run
dbname = 'dgsbycvs'
user = 'dgsbycvs'  # ElephantSQL chooses to reuse dbname and username
password = '_UGGyAcve_raJ3ErUvPykxWs5DGRYE7t'
host = 'ruby.db.elephantsql.com'  # Port is default 5432

In [ ]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [ ]:
pg_conn

<connection object at 0x7f32cf3d38d0; dsn: 'user=dgsbycvs password=xxx dbname=dgsbycvs host=ruby.db.elephantsql.com', closed: 0>

In [ ]:
dir(pg_conn)

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'async',
 'async_',
 'autocommit',
 'binary_types',
 'cancel',
 'close',
 'closed',
 'commit',
 'cursor',
 'cursor_factory',
 'deferrable',
 'dsn',
 'encoding',
 'fileno',
 'get_backend_pid',
 'get_dsn_parameters',
 'get_native_connection',
 'get_parameter_status',
 'get_transaction_status',
 'info',
 'isexecuting',
 'isolation_level',
 'lobject',
 'notices',
 'notifies',
 'pgconn_ptr',
 'poll',
 'protocol_version',
 'readonly',
 'reset',
 'rollback',
 'server_version

In [ ]:
pg_curs = pg_conn.cursor()  # Works the same as SQLite!

In [ ]:
help(pg_curs.execute)

Help on built-in function execute:

execute(...) method of psycopg2.extensions.cursor instance
    execute(query, vars=None) -- Execute query with bound vars.



In [ ]:
help(pg_curs.executemany)

Help on built-in function executemany:

executemany(...) method of psycopg2.extensions.cursor instance
    executemany(query, vars_list) -- Execute many queries with bound vars.



In [ ]:
# We're connected, but db is empty
# Let's run a simple example to populate (from the tk)
create_table_statement = """
CREATE TABLE test_table (
  id SERIAL PRIMARY KEY,
  name varchar(40) NOT NULL,
  data JSONB
);
"""
# NOTE - these types are PostgreSQL specific. This won't work in SQLite!

pg_curs.execute(create_table_statement)
pg_conn.commit()  # "Save" by committing

In [ ]:
insert_statement = """
INSERT INTO test_table (name, data) VALUES
(
  'A row name',
  null
),
(
  'Another row, with JSON this time',
  '{ "a": 1, "b": ["dog", "cat", 42], "c": true }'::JSONB
)
"""

pg_curs.execute(insert_statement)
pg_conn.commit()

In [ ]:
query = 'SELECT * FROM test_table;'
pg_curs.execute(query)

In [ ]:
pg_curs

<cursor object at 0x7f32cf489dd8; closed: 0>

In [ ]:
pg_curs.fetchall()

[(1, 'A row name', None),
 (2,
  'Another row, with JSON this time',
  {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [ ]:
# Database constraints are enforced - this is a good thing!
# Helps ensure data quality
pg_curs.execute('INSERT INTO test_table (name, data) VALUES (null, null);')

NotNullViolation: ignored

# ETL

Extract - Transform - Load

We'd like to get the RPG data *out* (extract) of SQLite and insert it into (load) PostgreSQL.

If we have to tweak it a bit between so it fits (or if we want to change it, e.g. calculate/summarize it), that's the transform step.

We're making our first "cloud" ETL!

In [ ]:
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2020-07-14 17:30:07--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-07-14 17:30:07--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-07-14 17:30:07--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-in

In [ ]:
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [ ]:
!ls

rpg_db.sqlite3	sample_data


In [ ]:
# Step 1 - Extract, getting data out of SQLite3
import sqlite3
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [ ]:
# Our goal - copy the charactercreator_character table
get_characters = 'SELECT * FROM charactercreator_character;'
characters = sl_curs.execute(get_characters).fetchall()

In [ ]:
len(characters)  # Correct number of characters

302

In [ ]:
characters[:5]

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [ ]:
# Step 1 complete! We have a list of tuples with all our character data
# Note - this is *not* a pandas dataframe
# We don't know types - so, for "Transform" we need to figure that out

# Step 2 - Transform
# Our goal is to make a schema to define a table that fits this data in Postgres
# Can we check the old schema?
sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [ ]:
# We need to make a create statement for PostgreSQL that captures these types
create_character_table = """
CREATE TABLE charactercreator_character (
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
);
"""

In [ ]:
# May need to rerun the .connect to refresh
pg_curs = pg_conn.cursor()
pg_curs.execute(create_character_table)
pg_conn.commit()

In [ ]:
# We can query tables if we want to check
# This is a clever optional thing, showing postgresql internals
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'dgsbycvs', None, True, False, False, False),
 ('public',
  'charactercreator_character',
  'dgsbycvs',
  None,
  True,
  False,
  False,
  False)]

In [ ]:
# We now have a place to insert our characters without needing to transform much
# Step 3 - Load!
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [ ]:
characters[0][1:]

('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [ ]:
example_insert = """
INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES """ + str(characters[0][1:]) + ";"

print(example_insert)  # Not running, just inspecting


INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES ('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1);


In [ ]:
# If we ran that, we'd insert the first character
# But we want them all - loops!
for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ";"
  pg_curs.execute(insert_character)

In [ ]:
# PostgreSQL cursor needs to fetch in separate step, unlike SQLite
pg_curs.execute('SELECT * FROM charactercreator_character LIMIT 5;')
pg_curs.fetchall()

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [ ]:
# It inserted, and we can query from our open cursor (because it did the insert)
# But other connections and cursors don't know about it yet - we didn't commit!
pg_conn.commit()

In [ ]:
# We've done a basic ETL! How can we verify?
len(characters)  # characters from SQLite

302

In [ ]:
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [ ]:
len(pg_characters)

302

In [ ]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [ ]:
# We could do more spot checks, but let's loop and check them all
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

In [ ]:
# No complaints - which means they're all the same!